In [1]:
import pandas as pd
import numpy as np

In [2]:
cust = pd.read_csv('customer_interactions.csv')

In [3]:
prod = pd.read_csv('product_details.csv', sep = ';').dropna(how='all', axis=1)
prod

,product_id,category,price,ratings
0,101,Electronics,500,4.5
1,102,Clothing,50,3.8
2,103,Home & Kitchen,200,4.2
3,104,Beauty,30,4.0
4,105,Electronics,800,4.8


In [4]:
hist = pd.read_csv('purchase_history.csv', sep = ';').dropna(how='all', axis=1)
hist

,customer_id,product_id,purchase_date
0,1,101,2023-01-01
1,1,105,2023-01-05
2,2,102,2023-01-02
3,3,103,2023-01-03
4,4,104,2023-01-04
5,5,101,2023-01-05


In [38]:
from faker import Faker
from datetime import date
from random import randint,seed

seed(2710)
fake = Faker()

dt = []
for i in range(30):
    dd = fake.date_between(date(2023, 1, 7), end_date=date(2023, 3, 10))
    
    dt.append(dd.strftime('%Y-%m-%d'))

userid = []
for i in range(30):
    userid.append(randint(1, 5))


prodid = []
for i in range(30):
    prodid.append(randint(101, 105))

d_sim = pd.DataFrame({'customer_id': userid, 'product_id': prodid, 'purchase_date': dt})

In [42]:
pd.concat([hist, d_sim], ignore_index = True).sort_values('product_id').to_csv('purchase_history_modify.csv')

',customer_id,product_id,purchase_date\n0,1,101,2023-01-01\n14,2,101,2023-02-26\n19,5,101,2023-02-10\n21,1,101,2023-01-12\n11,3,101,2023-02-08\n18,3,101,2023-03-08\n5,5,101,2023-01-05\n6,4,102,2023-03-03\n26,5,102,2023-01-08\n15,5,102,2023-02-10\n23,1,102,2023-01-10\n2,2,102,2023-01-02\n32,5,102,2023-02-26\n22,3,103,2023-01-21\n28,3,103,2023-02-14\n31,5,103,2023-02-19\n25,1,103,2023-01-08\n35,2,103,2023-03-05\n16,1,103,2023-02-22\n13,3,103,2023-02-18\n3,3,103,2023-01-03\n12,4,103,2023-02-06\n9,4,103,2023-01-10\n4,4,104,2023-01-04\n27,2,104,2023-03-04\n7,4,104,2023-01-31\n10,3,104,2023-02-02\n8,5,104,2023-01-10\n34,2,105,2023-01-13\n24,3,105,2023-03-02\n29,1,105,2023-02-19\n30,5,105,2023-01-31\n20,3,105,2023-02-11\n1,1,105,2023-01-05\n33,4,105,2023-02-09\n17,1,105,2023-02-22\n'

In [11]:
mrg = hist.merge(prod, on = 'product_id').merge(cust, on = 'customer_id')
mrg

,customer_id,product_id,purchase_date,category,price,ratings,page_views,time_spent
0,1,101,2023-01-01,Electronics,500,4.5,25,120
1,1,105,2023-01-05,Electronics,800,4.8,25,120
2,5,101,2023-01-05,Electronics,500,4.5,22,110
3,2,102,2023-01-02,Clothing,50,3.8,20,90
4,3,103,2023-01-03,Home & Kitchen,200,4.2,30,150
5,4,104,2023-01-04,Beauty,30,4.0,15,80


In [153]:
ftr = mrg.groupby(['product_id'])[['price', 'ratings', 'page_views', 'time_spent']].mean().reset_index()

In [216]:
#hist.pivot()

In [200]:
res = []
from sklearn.metrics.pairwise import cosine_similarity
for i in ftr.product_id.unique():
    for j in ftr.product_id.unique():
        if i == j:
            continue
        cos = cosine_similarity(ftr[ftr.product_id == i], ftr[ftr.product_id == j])
        res.append([i, j, cos[0][0]])


In [206]:
sim_df = pd.DataFrame(res, columns = ['product_id', 'product_recommend',
                                      'similarity']).sort_values(['product_id', 'similarity'], ascending = [True, False])

In [253]:
def get_recommendation(id):
    last_prod_id = hist[hist.customer_id == id].sort_values(['customer_id',
                                            'purchase_date'], ascending = [True, False]).drop_duplicates('customer_id').product_id.values[0]
    res = sim_df[sim_df.product_id == last_prod_id]
    return res.drop('product_id', axis = 1).to_json(orient = 'records')

In [254]:
get_recommendation(1)

'[{"product_recommend":101,"similarity":0.994874542},{"product_recommend":103,"similarity":0.853786715},{"product_recommend":102,"similarity":0.5193194791},{"product_recommend":104,"similarity":0.4061831346}]'

In [136]:
cnt_hist = hist.groupby(['product_id'])['purchase_date'].count().reset_index()
cnt_hist.rename(columns = {'purchase_date': 'purchase_number'}, inplace = True)
# Transform the table
hist.merge(cnt_hist, on = 'product_id')
#piv_hist = cnt_hist.pivot(index='customer_id' , columns='product_id', values='purchase_date').reset_index().fillna(0)
# Inspect the transformed table

,customer_id,product_id,purchase_date,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,purchase_number
0,1,101,2023-01-01,NaN,NaN,NaN,NaN,2
1,5,101,2023-01-05,NaN,NaN,NaN,NaN,2
2,1,105,2023-01-05,NaN,NaN,NaN,NaN,1
3,2,102,2023-01-02,NaN,NaN,NaN,NaN,1
4,3,103,2023-01-03,NaN,NaN,NaN,NaN,1
5,4,104,2023-01-04,NaN,NaN,NaN,NaN,1


In [127]:
cnt_hist

,product_id,purchase_date
0,101,2
1,102,1
2,103,1
3,104,1
4,105,1


In [128]:
piv_hist_cust = cust.merge(piv_hist, on = 'customer_id')
piv_hist_cust

,customer_id,page_views,time_spent,101,102,103,104,105
0,1,25,120,1.0,0.0,0.0,0.0,1.0
1,2,20,90,0.0,1.0,0.0,0.0,0.0
2,3,30,150,0.0,0.0,1.0,0.0,0.0
3,4,15,80,0.0,0.0,0.0,1.0,0.0
4,5,22,110,1.0,0.0,0.0,0.0,0.0


In [119]:
cust_x_prod = hist[['customer_id', 'product_id']].merge(prod[['product_id','price','ratings']], on = 'product_id')

In [120]:
feat = cust_x_prod.merge(piv_hist_cust, on = 'customer_id')
feat.columns = feat.columns.astype(str)

In [80]:
X = feat[['price', 'page_views', 'time_spent', '101', '102', '103', '104', '105']]
y = feat['ratings']

In [56]:
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

In [82]:
user_knn = KNeighborsRegressor(metric='cosine', n_neighbors=5)


In [83]:
user_knn.fit(X, y)


KNeighborsRegressor(metric='cosine')

In [88]:
pred = user_knn.predict(X)

In [89]:
pred

array([4.36, 4.36, 4.36, 4.2 , 4.36, 4.2 ])

In [90]:
feat['pred'] = pred

In [93]:
X

,price,page_views,time_spent,101,102,103,104,105
0,500,25,120,1.0,0.0,0.0,0.0,1.0
1,800,25,120,1.0,0.0,0.0,0.0,1.0
2,500,22,110,1.0,0.0,0.0,0.0,0.0
3,50,20,90,0.0,1.0,0.0,0.0,0.0
4,200,30,150,0.0,0.0,1.0,0.0,0.0
5,30,15,80,0.0,0.0,0.0,1.0,0.0


In [109]:
from sklearn.metrics.pairwise import cosine_similarity


for i in 
cosine_similarity(feat.drop(['customer_id', 'product_id'], axis = 1).iloc[0].values.reshape(1, -1), 
feat.drop(['customer_id', 'product_id'], axis = 1).iloc[5].values.reshape(1, -1))

array([[0.55873862]])

In [110]:
X

,price,page_views,time_spent,101,102,103,104,105
0,500,25,120,1.0,0.0,0.0,0.0,1.0
1,800,25,120,1.0,0.0,0.0,0.0,1.0
2,500,22,110,1.0,0.0,0.0,0.0,0.0
3,50,20,90,0.0,1.0,0.0,0.0,0.0
4,200,30,150,0.0,0.0,1.0,0.0,0.0
5,30,15,80,0.0,0.0,0.0,1.0,0.0


In [111]:
feat.groupby('product_id')[]

,customer_id,product_id,price,ratings,page_views,time_spent,101,102,103,104,105,pred
0,1,101,500,4.5,25,120,1.0,0.0,0.0,0.0,1.0,4.36
1,1,105,800,4.8,25,120,1.0,0.0,0.0,0.0,1.0,4.36
2,5,101,500,4.5,22,110,1.0,0.0,0.0,0.0,0.0,4.36
3,2,102,50,3.8,20,90,0.0,1.0,0.0,0.0,0.0,4.20
4,3,103,200,4.2,30,150,0.0,0.0,1.0,0.0,0.0,4.36
5,4,104,30,4.0,15,80,0.0,0.0,0.0,1.0,0.0,4.20


In [137]:
X

,price,page_views,time_spent,101,102,103,104,105
0,500,25,120,1.0,0.0,0.0,0.0,1.0
1,800,25,120,1.0,0.0,0.0,0.0,1.0
2,500,22,110,1.0,0.0,0.0,0.0,0.0
3,50,20,90,0.0,1.0,0.0,0.0,0.0
4,200,30,150,0.0,0.0,1.0,0.0,0.0
5,30,15,80,0.0,0.0,0.0,1.0,0.0
